In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
os.chdir("/sfs/qumulo/qhome/jtb3sud/PARCv2")
import PARC.data.data_meta_learning as data
#from parc import misc, metrics, visualization
from PARC.model import model_meta_learning as model


2024-05-08 18:18:12.565941: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-08 18:18:19.588005: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1284 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:01:00.0, compute capability: 8.6


# Data pipeline

In [2]:
# Get data and normalization
state_seq_whole, vel_seq_whole = data.clip_raw_data(sequence_length = 52, n_state_var = 3, purpose = "diff_training", folder_path = '/scratch/jtb3sud/meta_data/tnt/training', image_size = (128, 208))
state_seq_norm = data.data_normalization(state_seq_whole,3)
vel_seq_norm = data.data_normalization(vel_seq_whole,2)

Processing 183118.npy
Processing 183135.npy
Processing 183127.npy
Processing 183134.npy
Processing 183115.npy
Processing 183132.npy
Processing 183114.npy
Processing 183123.npy
Processing 183140.npy
Processing 183130.npy
Processing 183133.npy
Processing 183129.npy
Processing 183111.npy
Processing 183105.npy
Processing 183102.npy
Processing 183113.npy
Processing 183137.npy
Processing 183136.npy
Processing 183119.npy
Processing 183144.npy
Processing 183110.npy
Processing 183112.npy
Processing 183122.npy
Processing 183104.npy
Processing 183128.npy
Processing 183141.npy
Processing 183145.npy
Processing 183103.npy
Processing 183107.npy
Processing 183106.npy
Processing 183121.npy
Processing 183108.npy
Processing 183142.npy
Processing 183109.npy
Processing 183125.npy


# Training


### Stage 1: Differentiator training

In [3]:
# Create tf.dataset
dataset_input = tf.data.Dataset.from_tensor_slices((state_seq_norm[0][:,:,:,:3],vel_seq_norm[0][:,:,:,:2]))
dataset_label = tf.data.Dataset.from_tensor_slices((state_seq_norm[0][:,:,:,-3:],vel_seq_norm[0][:,:,:,-2:]))
dataset = tf.data.Dataset.zip((dataset_input, dataset_label))
dataset = dataset.shuffle(buffer_size = 2192) 
dataset = dataset.batch(8)

In [11]:
print(state_seq_whole.shape, vel_seq_whole.shape)

(339, 128, 208, 156) (339, 128, 208, 104)


In [9]:
print(state_seq_norm[0].shape, vel_seq_norm[0].shape)

(339, 128, 208, 156) (339, 128, 208, 104)


In [4]:
dataset

<_BatchDataset element_spec=((TensorSpec(shape=(None, 128, 208, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 128, 208, 2), dtype=tf.float64, name=None)), (TensorSpec(shape=(None, 128, 208, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 128, 208, 2), dtype=tf.float64, name=None)))>

In [6]:
tf.keras.backend.clear_session()
# step size needs to be changed to 1/total number of steps
#parc = model.PARCv2(n_state_var = 3, n_time_step = 1, step_size= 1/60, solver = "rk4", mode = "differentiator_training")
parc = model.PARCv2(n_state_var = 3, n_time_step = 52, step_size= 1/52, solver = "rk4", mode = "differentiator_training", image_size = (128, 208))
parc.differentiator.load_weights('/sfs/qumulo/qhome/jtb3sud/PARCv2/Pretrained_Weights/em/parc2_diff_rk4.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001, beta_1 = 0.9, beta_2 = 0.999))
parc.fit(dataset, epochs = 100, shuffle = True)

Epoch 1/100


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/sfs/qumulo/qhome/jtb3sud/PARCv2/PARC/model/model_meta_learning.py", line 156, in train_step  *
        total_loss  = (tf.keras.losses.MeanAbsoluteError(reduction = 'sum')(state_pred,state_var_gt) +
    File "/usr/local/lib/python3.10/dist-packages/keras/src/losses.py", line 142, in __call__  **
        losses = call_fn(y_true, y_pred)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/losses.py", line 268, in call  **
        return ag_fn(y_true, y_pred, **self._fn_kwargs)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/losses.py", line 1749, in mean_absolute_error
        return backend.mean(tf.abs(y_pred - y_true), axis=-1)

    ValueError: Dimensions must be equal, but are 3 and 156 for '{{node mean_absolute_error/sub}} = Sub[T=DT_FLOAT](Cast_2, concatenate_1/concat)' w

In [6]:
parc.differentiator.save_weights('/sfs/qumulo/qhome/jtb3sud/PARCv2/Pretrained_Weights/tnt/parc2_diff_rk4.h5')

### Stage 2: Data-driven integration training

In [ ]:
# Pretrain integrator
tf.keras.backend.clear_session()
parc = model.PARCv2(n_state_var = 3, n_time_step = 1, step_size= 1/60, solver = "rk4", mode = "integrator_training")
parc.integrator.load_weights('/sfs/qumulo/qhome/jtb3sud/PARCv2/Pretrained_Weights/em/parc2_int_rk4.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001, beta_1 = 0.9, beta_2 = 0.999))
parc.fit(dataset, epochs = 125, shuffle = True)

Epoch 1/125


2024-04-30 11:52:17.496255: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2024-04-30 11:52:20.861035: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55c617b17e00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-30 11:52:20.861077: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2024-04-30 11:52:20.974633: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-30 11:52:21.937055: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


232/232 [==============================] - 154s 498ms/step - total_loss: 686.6307
Epoch 2/125
232/232 [==============================] - 115s 494ms/step - total_loss: 567.7000
Epoch 3/125
232/232 [==============================] - 115s 494ms/step - total_loss: 545.1809
Epoch 4/125
232/232 [==============================] - 115s 494ms/step - total_loss: 523.2788
Epoch 5/125
232/232 [==============================] - 115s 494ms/step - total_loss: 514.4716
Epoch 6/125
232/232 [==============================] - 115s 494ms/step - total_loss: 504.3910
Epoch 7/125
232/232 [==============================] - 115s 494ms/step - total_loss: 494.5680
Epoch 8/125
232/232 [==============================] - 115s 494ms/step - total_loss: 488.8584
Epoch 9/125
232/232 [==============================] - 115s 494ms/step - total_loss: 486.2922
Epoch 10/125
232/232 [==============================] - 115s 494ms/step - total_loss: 484.7857
Epoch 11/125
232/232 [==============================] - 115s 494ms/step

In [ ]:
parc.integrator.save_weights('/sfs/qumulo/qhome/jtb3sud/PARCv2/Pretrained_Weights/tnt/parc2_int_rk4.h5')